In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [89]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [90]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [91]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [92]:
# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')
# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [93]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  channelGrouping       5 non-null      object
 14  socialEngagementType  5 non-null      object
dtypes: Int64(4), object(11)
memory usage: 752.0+

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [94]:
# Get format schema nested field

def format_schema_field(schema_field, indent=0):
    indent_str = "  " * indent
    field_info = f"{indent_str}{schema_field.name} ({schema_field.field_type})"
    
    if schema_field.mode != "NULLABLE":
        field_info += f" - {schema_field.mode}"
    
    if schema_field.description:
        field_info += f" - {schema_field.description}"
    
    nested_indent = indent + 2
    if schema_field.field_type == "RECORD":
        for sub_field in schema_field.fields:
            field_info += "\n" + format_schema_field(sub_field, nested_indent)
    
    return field_info


table_ref = dataset_ref.table('ga_sessions_20160801')
table = client.get_table(table_ref)

# Display schemas
print("SCHEMA field for the 'totals' column:\n")
print(format_schema_field(table.schema[5]))
print()

print("\nSCHEMA field for the 'trafficSource' column:\n")
print(format_schema_field(table.schema[6]))
print()

print("\nSCHEMA field for the 'device' column:\n")
print(format_schema_field(table.schema[7]))
print()

print("\nSCHEMA field for the 'geoNetwork' column:\n")
print(format_schema_field(table.schema[8]))
print()

print("\nSCHEMA field for the 'customDimensions' column:\n")
print(format_schema_field(table.schema[9]))
print()

print("\nSCHEMA field for the 'hits' column:\n")
print(format_schema_field(table.schema[10]))

SCHEMA field for the 'totals' column:

totals (RECORD)
    visits (INTEGER)
    hits (INTEGER)
    pageviews (INTEGER)
    timeOnSite (INTEGER)
    bounces (INTEGER)
    transactions (INTEGER)
    transactionRevenue (INTEGER)
    newVisits (INTEGER)
    screenviews (INTEGER)
    uniqueScreenviews (INTEGER)
    timeOnScreen (INTEGER)
    totalTransactionRevenue (INTEGER)
    sessionQualityDim (INTEGER)


SCHEMA field for the 'trafficSource' column:

trafficSource (RECORD)
    referralPath (STRING)
    campaign (STRING)
    source (STRING)
    medium (STRING)
    keyword (STRING)
    adContent (STRING)
    adwordsClickInfo (RECORD)
        campaignId (INTEGER)
        adGroupId (INTEGER)
        creativeId (INTEGER)
        criteriaId (INTEGER)
        page (INTEGER)
        slot (STRING)
        criteriaParameters (STRING)
        gclId (STRING)
        customerId (INTEGER)
        adNetworkType (STRING)
        targetingCriteria (RECORD)
            boomUserlistId (INTEGER)
        isV

In [95]:
# Create data table from 'totals'

query = """
SELECT
  product.v2ProductName as product,
  SUM(totals.hits) as total_hits,
  SUM(totals.visits) as total_visits,
  SUM(totals.pageviews) as total_page_views,
  SUM(totals.newVisits) as total_first_visits,
  SUM(totals.timeOnScreen) as total_time_on_screen,
  AVG(totals.timeOnScreen) as avg_time_on_screen,
  SUM(totals.timeOnSite) as total_time_on_site,
  AVG(totals.timeOnSite) as avg_time_on_site,
  SUM(totals.screenviews) as total_screen_views,
  SUM(totals.transactions) as total_transactions,
  SUM(totals.transactionRevenue) as total_revenue,
  AVG(totals.transactionRevenue) as avg_revenue,
  SUM(totals.uniqueScreenviews) as total_unique_screen_views,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) product
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
  (geoNetwork.country = 'United States')
GROUP BY product
"""

df_totals = client.query(query).to_dataframe()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product                    522 non-null    object 
 1   total_hits                 522 non-null    Int64  
 2   total_visits               522 non-null    Int64  
 3   total_page_views           522 non-null    Int64  
 4   total_first_visits         516 non-null    Int64  
 5   total_time_on_screen       0 non-null      Int64  
 6   avg_time_on_screen         0 non-null      float64
 7   total_time_on_site         522 non-null    Int64  
 8   avg_time_on_site           522 non-null    float64
 9   total_screen_views         0 non-null      Int64  
 10  total_transactions         507 non-null    Int64  
 11  total_revenue              507 non-null    Int64  
 12  avg_revenue                507 non-null    float64
 13  total_unique_screen_views  0 non-null      Int64  

,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_screen,avg_time_on_screen,total_time_on_site,avg_time_on_site,total_screen_views,total_transactions,total_revenue,avg_revenue,total_unique_screen_views
0,Waze Mood Original Window Decal,28252,652,19190,520,<NA>,NaN,584654,896.708589,<NA>,278,18230450000,6.557716e+07,<NA>
1,Large Zipper Top Tote Bag,92977,1833,64923,736,<NA>,NaN,2233626,1218.563011,<NA>,533,83319600000,1.624164e+08,<NA>
2,Google Sunglasses,1125318,19681,802939,8650,<NA>,NaN,26239500,1333.443439,<NA>,11170,2686190590000,2.595102e+08,<NA>
3,Google Infant Zip Hood Pink,73973,1701,50550,903,<NA>,NaN,1770411,1043.259281,<NA>,833,146583650000,1.827726e+08,<NA>
4,Waze Dress Socks,9347,422,7509,213,<NA>,NaN,279390,662.061611,<NA>,168,9613500000,5.722321e+07,<NA>


In [96]:
df_totals = df_totals.dropna(axis=1, how='all').fillna(0)

print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product             522 non-null    object 
 1   total_hits          522 non-null    Int64  
 2   total_visits        522 non-null    Int64  
 3   total_page_views    522 non-null    Int64  
 4   total_first_visits  522 non-null    Int64  
 5   total_time_on_site  522 non-null    Int64  
 6   avg_time_on_site    522 non-null    float64
 7   total_transactions  522 non-null    Int64  
 8   total_revenue       522 non-null    Int64  
 9   avg_revenue         522 non-null    float64
dtypes: Int64(7), float64(2), object(1)
memory usage: 44.5+ KB
None


,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue
0,Waze Mood Original Window Decal,28252,652,19190,520,584654,896.708589,278,18230450000,6.557716e+07
1,Large Zipper Top Tote Bag,92977,1833,64923,736,2233626,1218.563011,533,83319600000,1.624164e+08
2,Google Sunglasses,1125318,19681,802939,8650,26239500,1333.443439,11170,2686190590000,2.595102e+08
3,Google Infant Zip Hood Pink,73973,1701,50550,903,1770411,1043.259281,833,146583650000,1.827726e+08
4,Waze Dress Socks,9347,422,7509,213,279390,662.061611,168,9613500000,5.722321e+07


In [107]:
df_totals.describe()

,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue
count,522.0,522.0,522.0,522.0,522.0,522.000000,522.0,522.0,5.220000e+02
mean,74844.770115,1563.716475,53580.714559,768.961686,1774360.208812,1248.482667,723.14751,152969453659.003845,2.677724e+08
std,108626.06239,2374.326617,78677.588678,1232.899993,2561553.147775,408.056154,1208.597645,269644715718.549133,3.887066e+08
min,12.0,2.0,10.0,0.0,172.0,86.000000,0.0,0.0,0.000000e+00
25%,14869.75,264.0,10529.25,104.0,323454.25,1004.207947,98.25,16168942500.0,1.266905e+08
50%,41024.5,821.5,28867.5,352.5,988410.5,1198.021880,350.0,76758200000.0,1.992690e+08
75%,83547.75,1821.5,60376.25,921.0,2123290.25,1436.866451,763.5,169877535000.0,2.895418e+08
max,1125318.0,19842.0,802939.0,10538.0,26239500.0,3876.000000,11384.0,2686190590000.0,6.826960e+09


In [97]:
query = """
SELECT
  product.v2ProductName as product,
  CASE 
    WHEN hits.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
    WHEN hits.eCommerceAction.action_type = '2' THEN 'Product detail views'
    WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
    WHEN hits.eCommerceAction.action_type = '6' THEN 'Completed purchase'
  END as action,
  trafficSource.medium as medium,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) product
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
    (geoNetwork.country = 'United States')
"""

df_hits_traffic = client.query(query).to_dataframe()
print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816260 entries, 0 to 816259
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   product  816260 non-null  object
 1   action   816260 non-null  object
 2   medium   816260 non-null  object
dtypes: object(3)
memory usage: 18.7+ MB
None


,product,action,medium
0,Google Doodle Decal,Click through of product lists,(none)
1,Google Doodle Decal,Product detail views,(none)
2,Google Doodle Decal,Click through of product lists,(none)
3,Android Men's Take Charge Short Sleeve Tee Purple,Check out,(none)
4,Android Men's Take Charge Short Sleeve Tee Purple,Check out,(none)


In [98]:
df_hits_traffic.drop(['medium', 'action'], axis=1)

,product
0,Google Doodle Decal
1,Google Doodle Decal
2,Google Doodle Decal
3,Android Men's Take Charge Short Sleeve Tee Purple
4,Android Men's Take Charge Short Sleeve Tee Purple
...,...
816255,Badge Holder
816256,Badge Holder
816257,Badge Holder
816258,Google Bluetooth Speaker-Power Bank


In [99]:
df_ohe = pd.get_dummies(df_hits_traffic[['medium', 'action']], dtype=int)
df_hit_traffic = pd.concat([df_hits_traffic.drop(['medium', 'action'], axis=1),
                            df_ohe], axis=1)

def modify_columns(array):
    temp = []
    for d in array:
        temp.append(d.replace('(', '').replace(')', '').replace(' ', '_'))
    return temp


df_hit_traffic.columns = modify_columns(np.array(df_hit_traffic.columns))

df_hit_traffic

,product,medium_none,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Google Doodle Decal,1,0,0,0,0,0,0,0,1,0,0
1,Google Doodle Decal,1,0,0,0,0,0,0,0,0,0,1
2,Google Doodle Decal,1,0,0,0,0,0,0,0,1,0,0
3,Android Men's Take Charge Short Sleeve Tee Purple,1,0,0,0,0,0,0,1,0,0,0
4,Android Men's Take Charge Short Sleeve Tee Purple,1,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
816255,Badge Holder,0,0,0,0,0,1,0,0,1,0,0
816256,Badge Holder,0,0,0,0,0,1,0,0,0,0,1
816257,Badge Holder,0,0,0,0,0,1,0,0,1,0,0
816258,Google Bluetooth Speaker-Power Bank,0,0,0,0,0,1,0,0,1,0,0


In [100]:
df_hit_traffic = df_hit_traffic.groupby('product').sum().reset_index()

print(df_hit_traffic.info())
df_hit_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   product                                522 non-null    object
 1   medium_none                            522 non-null    int32 
 2   medium_not_set                         522 non-null    int32 
 3   medium_affiliate                       522 non-null    int32 
 4   medium_cpc                             522 non-null    int32 
 5   medium_cpm                             522 non-null    int32 
 6   medium_organic                         522 non-null    int32 
 7   medium_referral                        522 non-null    int32 
 8   action_Check_out                       522 non-null    int32 
 9   action_Click_through_of_product_lists  522 non-null    int32 
 10  action_Completed_purchase              522 non-null    int32 
 11  action_Product_deta

,product,medium_none,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,1 oz Hand Sanitizer,2027,0,4,97,24,593,109,1113,708,398,635
1,15 oz Ceramic Mug,16,0,0,2,2,8,0,1,15,0,12
2,16 oz. Hot and Cold Tumbler,324,0,6,17,3,98,24,131,167,40,134
3,16 oz. Hot/Cold Tumbler,160,0,0,22,8,62,8,73,92,16,79
4,20 oz Stainless Steel Insulated Tumbler,3754,0,10,176,57,1275,202,956,2302,324,1892


In [101]:
df = pd.merge(df_totals, df_hit_traffic, how='outer', on='product')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 0 to 521
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   product                                522 non-null    object 
 1   total_hits                             522 non-null    Int64  
 2   total_visits                           522 non-null    Int64  
 3   total_page_views                       522 non-null    Int64  
 4   total_first_visits                     522 non-null    Int64  
 5   total_time_on_site                     522 non-null    Int64  
 6   avg_time_on_site                       522 non-null    float64
 7   total_transactions                     522 non-null    Int64  
 8   total_revenue                          522 non-null    Int64  
 9   avg_revenue                            522 non-null    float64
 10  medium_none                            522 non-null    int32  
 11  medium

,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue,...,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Waze Mood Original Window Decal,28252,652,19190,520,584654,896.708589,278,18230450000,6.557716e+07,...,0,8,10,0,53,119,261,157,88,146
1,Large Zipper Top Tote Bag,92977,1833,64923,736,2233626,1218.563011,533,83319600000,1.624164e+08,...,2,2,67,7,276,74,255,858,76,644
2,Google Sunglasses,1125318,19681,802939,8650,26239500,1333.443439,11170,2686190590000,2.595102e+08,...,0,70,712,380,4420,608,7833,5130,2596,4122
3,Google Infant Zip Hood Pink,73973,1701,50550,903,1770411,1043.259281,833,146583650000,1.827726e+08,...,0,0,15,39,198,31,448,576,152,525
4,Waze Dress Socks,9347,422,7509,213,279390,662.061611,168,9613500000,5.722321e+07,...,0,3,6,4,104,66,164,116,50,92


In [102]:
X = df.drop('product', axis=1)

In [103]:
from sklearn.ensemble import IsolationForest

model = IsolationForest()
model.fit(X)

IsolationForest()

In [117]:
df_result = df.copy()

# Result
result = model.predict(X)

df_result['anomaly'] = result
print(df_result.info())
df_result.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 0 to 521
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   product                                522 non-null    object 
 1   total_hits                             522 non-null    Int64  
 2   total_visits                           522 non-null    Int64  
 3   total_page_views                       522 non-null    Int64  
 4   total_first_visits                     522 non-null    Int64  
 5   total_time_on_site                     522 non-null    Int64  
 6   avg_time_on_site                       522 non-null    float64
 7   total_transactions                     522 non-null    Int64  
 8   total_revenue                          522 non-null    Int64  
 9   avg_revenue                            522 non-null    float64
 10  medium_none                            522 non-null    int32  
 11  medium

,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue,...,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views,anomaly
0,Waze Mood Original Window Decal,28252,652,19190,520,584654,896.708589,278,18230450000,6.557716e+07,...,8,10,0,53,119,261,157,88,146,1
1,Large Zipper Top Tote Bag,92977,1833,64923,736,2233626,1218.563011,533,83319600000,1.624164e+08,...,2,67,7,276,74,255,858,76,644,1
2,Google Sunglasses,1125318,19681,802939,8650,26239500,1333.443439,11170,2686190590000,2.595102e+08,...,70,712,380,4420,608,7833,5130,2596,4122,-1
3,Google Infant Zip Hood Pink,73973,1701,50550,903,1770411,1043.259281,833,146583650000,1.827726e+08,...,0,15,39,198,31,448,576,152,525,1
4,Waze Dress Socks,9347,422,7509,213,279390,662.061611,168,9613500000,5.722321e+07,...,3,6,4,104,66,164,116,50,92,1


In [120]:
df_result = df.copy()

# Result
result = model.predict(X)

df_result['anomaly'] = result
df_result['anomaly'] = df_result['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [121]:
# Scatter plot total_visits vs total_transactions
import plotly.express as px

fig = px.scatter(
    x=df_result['total_visits'], 
    y=df_result['total_transactions'],
    color=df_result['anomaly'],
    title="Sample Scatter Plot",
    labels={'x': 'Total Visits', 'y': 'Total Transactions'}
)

# Show the plot
fig.show()

In [122]:
df_anomaly = df_result[df_result['anomaly'] == 'Anomaly']
print(df_anomaly.info())
df_anomaly.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 2 to 515
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   product                                43 non-null     object 
 1   total_hits                             43 non-null     Int64  
 2   total_visits                           43 non-null     Int64  
 3   total_page_views                       43 non-null     Int64  
 4   total_first_visits                     43 non-null     Int64  
 5   total_time_on_site                     43 non-null     Int64  
 6   avg_time_on_site                       43 non-null     float64
 7   total_transactions                     43 non-null     Int64  
 8   total_revenue                          43 non-null     Int64  
 9   avg_revenue                            43 non-null     float64
 10  medium_none                            43 non-null     int32  
 11  medium_

,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue,...,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views,anomaly
2,Google Sunglasses,1125318,19681,802939,8650,26239500,1333.443439,11170,2686190590000,2.595102e+08,...,70,712,380,4420,608,7833,5130,2596,4122,Anomaly
19,Leatherette Journal,266681,5633,191325,2588,5975295,1061.519808,2970,970634050000,3.554134e+08,...,18,150,103,977,143,1714,1883,580,1456,Anomaly
88,Windup Android,285855,5982,205003,2707,7090244,1186.453146,3257,496983260000,1.939069e+08,...,50,171,77,1473,268,1809,1931,602,1640,Anomaly
101,Red Shine 15 oz Mug,324585,6857,235309,3088,8229784,1200.201838,3736,499910570000,1.489159e+08,...,14,157,83,1389,163,2108,2190,704,1855,Anomaly
103,Electronics Accessory Pouch,273702,4567,193730,2334,6551249,1436.362421,1736,537528120000,3.524775e+08,...,2,185,60,1201,193,908,1903,296,1460,Anomaly


In [123]:
# Scatter plot total_visits vs total_transactions

fig = px.scatter(
    x=df_anomaly['total_visits'], 
    y=df_anomaly['total_transactions'], 
    text=df_anomaly['product'], 
    title="Sample Scatter Plot",
    labels={'x': 'Total Visits', 'y': 'Total Transactions'}
)

# Show the plot
fig.show()

In [125]:
df_anomaly[['product', 'total_visits', 'total_first_visits', 'avg_time_on_site', 'total_transactions', 'total_revenue']].to_csv('./result/anomaly_products.csv', index=False)

In [126]:
df_anomaly['product'].values

array(['Google Sunglasses', 'Leatherette Journal', 'Windup Android',
       'Red Shine 15 oz Mug', 'Electronics Accessory Pouch',
       'YouTube Custom Decals',
       "Google Women's Short Sleeve Hero Tee White",
       'Google Laptop Backpack', 'Maze Pen',
       'Google Water Resistant Bluetooth Speaker', 'Recycled Mouse Pad',
       'YouTube Twill Cap',
       "Google Men's Short Sleeve Badge Tee Charcoal", 'Google Twill Cap',
       "Google Men's  Zip Hoodie",
       "Google Men's 100% Cotton Short Sleeve Hero Tee Red",
       "Google Women's Fleece Hoodie",
       "YouTube Men's Short Sleeve Hero Tee Black",
       'Google 22 oz Water Bottle',
       'Google Laptop and Cell Phone Stickers', 'Badge Holder',
       "Google Women's Performance Polo Grey/Black",
       "Google Men's 100% Cotton Short Sleeve Hero Tee Navy",
       'Google Rucksack', 'Google Alpine Style Backpack',
       'Engraved Ceramic Google Mug',
       "Google Men's Vintage Badge Tee Black",
       "Google Men'